In [4]:
#make imports

import os
import sys
import pandas as pd
import numpy as np
import re
import nltk
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

#method to preprocess the text
def preprocess_text(text):

    #remove the html tags / links
    text = re.sub(r'<.*?>', '', text)

    #remove special characters
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    #lemmatize the text
    lemmatizer = WordNetLemmatizer()
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

    #normalize the text
    text = text.lower()

    return text

#load the saved model
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.load_state_dict(torch.load("/kaggle/input/inference_model/pytorch/inference_slug/1/finetuned_final_final.pth"), strict=False)

#take input from user
input_text = input("Enter the text: ")

#preprocess the input text
input_text = preprocess_text(input_text)

#send model to gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

#generate summary
model.eval()

def generate_summary(review_text):
    # tokenize input review
    inputs = tokenizer.encode(review_text+" TL;DR ", return_tensors='pt').to(device)

    # generate summary
    with torch.no_grad():
        summary_ids = model.generate(inputs, max_length=len(inputs[0])+31, num_beams=4, repetition_penalty=3.0, length_penalty=3.0, early_stopping=False)
    
    # decode summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

gen_summary = generate_summary(input_text).split("TL;DR")[-1].strip()
print("The generated summary is: ", gen_summary)

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/co

Enter the text:  If you are looking for the secret ingredient in Robitussin I believe I have found it.  I got this in addition to the Root Beer Extract I ordered (which was good) and made some cherry soda.  The flavor is very medicinal.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The generated summary is:  best greatum a treat coffeenut but love taste gluten my dog not excellent tea free ofy likeummy snacktt food gch tasty yuffs product
